In [15]:
import websockets
import asyncio
import requests
import os
from dotenv import load_dotenv
from datetime import datetime
from pydub import AudioSegment
from pydub.utils import which
from pydub.playback import play
import io
import json

AudioSegment.converter = which("ffmpeg")  # Set ffmpeg path
AudioSegment.ffprobe = which("ffprobe") 
load_dotenv()

True

In [8]:


# send auth post request 

PLAYHT_API_KEY = os.getenv("PLAYHT_API_KEY")
PLAYHT_USER_ID = os.getenv("PLAYHT_USER_ID")

def auth():
    url = "https://api.play.ht/api/v4/websocket-auth"
    headers = {
        "Authorization": "Bearer " + PLAYHT_API_KEY,
        "X-User-Id"    : PLAYHT_USER_ID,
        "Content-Type" : "application/json"
        }
    
    response = requests.post(url, headers=headers)

    return response.json()


response = auth()

In [ ]:
async def handle_request(text, model):
    # takes input text, passes it to tts model, receives audio file over websocket
    
    global response

    expiry_date = datetime.strptime(response["expires_at"], "%Y-%m-%dT%H:%M:%S.%fZ")
    current_date = datetime.now()
    # if time diff is less than an hour, renew token
    if (expiry_date - current_date).seconds < 3600:
        response = auth()

    url = response['websocket_urls'][model]

    request = {
         "text": text, # less than 20k characters
         "voice": "s3://voice-cloning-zero-shot/1806c2a9-2c23-4336-93a5-5447b80295e9/original/manifest.json", #GordonRamsay clone
         "output_format": "mp3",
         "temperature": "0.7",
         "speed": "1.0",
    }

    audio_chunks = []

    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps(request))
        while True:
            message = await websocket.recv()

            if isinstance(message, bytes):
                    # Received audio chunk
                    audio_chunks.append(message)
            else:
                # Received a JSON message
                data = json.loads(message)

                if data.get("type") == "start":
                    print(f"Processing started. Request ID: {data.get('request_id')}")

                elif data.get("type") == "end":
                    print(f"Processing ended. Request ID: {data.get('request_id')}")
                    break

        # Combine received audio chunks
        audio_data = b"".join(audio_chunks)

        # DEBUG: Save raw audio file to check format
        with open("output_raw.mp3", "wb") as f:
            f.write(audio_data)
        print("Raw audio saved as output_raw.mp3")

        # Convert binary data into an audio segment
        audio_segment = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")

        # Save or play the audio
        audio_segment.export("output.mpeg", format="mp3")

        # Play the audio
        play(audio_segment)
        print("Audio saved as output.mpeg")


In [46]:
response['websocket_urls'].keys()
model = "PlayDialog"

In [48]:
text = """You're such a fucking twat, you should've been aborted."""

await handle_request(text, model)
    

Processing started. Request ID: e2971473-1403-427b-a873-48a422e6dd5f
Processing ended. Request ID: e2971473-1403-427b-a873-48a422e6dd5f
Raw audio saved as output_raw.mp3
Audio saved as output.mpeg


In [ ]:
import requests

headers = {
        "Authorization": "Bearer " + PLAYHT_API_KEY,
        "X-User-Id"    : PLAYHT_USER_ID,
        "Content-Type" : "application/json"
        }

url = "https://api.play.ht/api/v2/cloned-voices"



response1 = requests.get(url, headers=headers)

print(response1.text)

[{"id":"s3://voice-cloning-zero-shot/1806c2a9-2c23-4336-93a5-5447b80295e9/original/manifest.json","name":"GordonRamsay","type":"instant","gender":"male","voice_engine":"PlayHT2.0"}]
